#### Course 6 Анализ тональности отзывов
#### Week 4 Демонстрация работы алгоритма
##### Теперь, когда у вас уже получается делать неплохие решения задачи сентимент-анализа, пора научиться собирать для вашего алгоритма интерактивную демонстрацию в виде простенькой веб-странички, чтобы на него могли посмотреть в действии не только те, кто может запустить ipython notebook.

Инструкция по выполнению

Часто оказывается полезно представить результаты вашей работы в виде интерактивной демонстрации, в которую можно будет вводить какие-то данные и получать результат работы вашей модели. В этом задании вам предлагается реализовать такую демонстрацию для вашего алгоритма анализа тональности отзывов на фильмы, используя flask.

Даже если вы никогда не сталкивались с веб-программированием и flask'ом - познакомиться с ним совершенно не сложно. Сделать первые шаги вам поможет прекрасный Викиучебник по Flask: https://ru.wikibooks.org/wiki/Flask

Разработку демки для вашей задачи существенно упростит архив кода для аналогичной демки в аттаче (с которым все равно проще разбираться после основ Flask): https://d3c33hcgiwev3.cloudfront.net/_0e0ae42bba09a7b92c3c0493a0423dd6_simple_demo.zip?Expires=1523404800&Signature=TjoSeam9eVWYJCSfkVGiduaRgEZzwaG7u6BvR-IS1QG9DV3jVZEzWG6nM8wK61yn-7t5hWg6RyEZkxVxWxnjMCgoE0qJ97NWso5Gf35~ahZGSwH8v83QxQpthq3mw8dI9QEqeJ3JOV86PYFzdWclGpkNDmS4F3YslTi-IXBEfXY_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A

В процессе вы столкнетесь с необходимостью сохранять в файл и загружать питоновский объект - обученный классификатор. Для этого вам пригодится библиотека pickle.

#### Импортируем отзывы:

In [1]:
import nltk

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
from nltk.corpus import movie_reviews

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline

#### Создаем списки файлов содержащих нег. и поз. отзывы:

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
" ".join(movie_reviews.words(fileids=posids[8])) # Test NEGATIVE review

u'after bloody clashes and independence won , lumumba refused to pander to the belgians , who continued a condescending and paternalistic relationship with the congo . their officers , particularly general janssens ( rudi delhem ) in the force publique , the congo \' s army , caused rebellions , undermining lumumba , who was outraged at the rape and murder of belgian nationals . with unrest building , moise tshombe ( pascal nzonzi ) and the province of katanga , which contained 70 percent of the country \' s resources , proclaimed secession . lumumba replaced janssens , making mobutu a colonel , and went on a pacification tour with congolese president joseph kasa vubu ( maka kotto ) , but it was too late . \' when you want to drown a dog , you say it has rabies , \' prophesies lumumba of his own fate . peck and bonitzer do an exemplary job telling a complicated tale with a myriad of players , although they frequently succumb to cliche , particularly regarding lumumba \' s private life 

In [4]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids] # Тексты целиком
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]
len(negfeats), len(posfeats)

(1000, 1000)

In [5]:
texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [11]:
clf_pipeline = Pipeline( [("vectorizer", CountVectorizer(ngram_range=(1, 2), analyzer='word')), 
                          ("classifier", LogisticRegression())] )
clf_pipeline.fit(texts, labels)
print (clf_pipeline)
auc1 = cross_val_score(clf_pipeline, texts, labels) #scoring='roc_auc'
print (auc1, auc1.mean())

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
     ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
(array([0.81137725, 0.84684685, 0.85435435]), 0.8375261489033944)


In [9]:
import pickle

In [13]:
with open('corpora_movie_reviews.pkl', 'wb') as f: 
    pickle.dump(clf_pipeline, f, protocol=2)   

In [ ]:
#Для тестирования

In [76]:
with open('corpora_movie_reviews.pkl', 'rb') as f: 
    clf_pipeline = pickle.load(f)

In [7]:
labels_test = clf_pipeline.predict(texts)
print (sum(labels == labels_test))

2000


In [12]:
print (clf_pipeline.predict(["Amazing film! I will advice it to all my friends. Genious",
                           "Awful film! The man who advised me to watch it is really crazy idiot."]))

[0 0]
